In [1]:
import warnings
import compare_MSAG_range as cmp
import geopandas as gpd
import os
import pandas as pd

In [2]:
gdf = cmp.getAddressPoints()[0]

In [3]:
gdf.fillna("", inplace=True)

In [4]:
gdf.STCODE.unique()

array(['RD', '', 'ST', 'LN', 'DR', 'CT', 'WAY', 'LP', 'AVE', 'BLVD',
       'TER', 'PL', 'CIR', 'HWY', 'PKWY', 'ALY', 'PIKE'], dtype=object)

In [5]:
gdf.DI.unique()

array(['', 'S', 'N', 'E', 'W', 'SW', 'NW'], dtype=object)

In [6]:
intrado = cmp.getIntrado()[2]

In [7]:
[esn for esn in gdf.ESN.unique() if esn not in intrado.ESN.unique()]

[39]

In [8]:
selectedCols = ['ESN', 'DI', 'STREET', 'STCODE', 'CTCODE']

In [9]:
gdf['KEY'] = gdf[selectedCols].apply(lambda row: str(row.ESN) + '_' + row.DI + '_' + row.STREET + '_' +  row.STCODE + '_' +  row.CTCODE, axis=1)

In [10]:
intrado['KEY'] = intrado[selectedCols].apply(lambda row: str(row.ESN) + '_' + row.DI + '_' + row.STREET + '_' +  row.STCODE + '_' +  row.CTCODE, axis=1)

In [11]:
len(intrado.KEY.unique())

6008

In [12]:
intrado.shape

(7973, 8)

In [13]:
commonKeys = [key for key in gdf.KEY.unique() if key in intrado.KEY.unique()]
len(commonKeys)

5667

In [14]:
len([key for key in gdf.KEY.unique() if key not in intrado.KEY.unique()])

278

In [15]:
len([key for key in intrado.KEY.unique() if key not in gdf.KEY.unique()])

341

In [16]:
def to_int(x):
    try:
        return int(x)
    except:
        return -1

In [17]:
def withinRange(x, ie_sdf, ESN=False, STCODE=False):
    for i in range(ie_sdf.shape[0]):
        try: 
            if ie_sdf.LOW.values[i] <= x <= ie_sdf.HIGH.values[i]:
                diff = 1
                if ESN:
                    match = 'Matched in one of the multiple ranges with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
                elif STCODE:
                    match = 'Matched in one of the multiple ranges with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
                else:
                    match = 'Matched in one of the multiple ranges'
                return diff, match
        except BaseException as e:
            print(e)
            print(x, ie_sdf.LOW.values[i], ie_sdf.HIGH.values[i])
            break
    if x > ie_sdf.HIGH.values[ie_sdf.shape[0]-1]:
        diff = 2
        if ESN:
            match = 'Outside of the multiple house number ranges in the high value with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
        elif STCODE:
            match = 'Outside of the multiple house number ranges in the high value with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
        else:
            match = 'Outside of the multiple house number ranges in the high value'
    elif x < ie_sdf.LOW.values[0]:
        diff = 3
        if ESN:
            match = 'Outside of the multiple house number ranges in the low value with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
        elif STCODE:
            match = 'Outside of the multiple house number ranges in the low value with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
        else:
            match = 'Outside of the multiple house number ranges in the low value'
    else:
        diff = 4
        if ESN:
            match = 'Outside of one of the multiple ranges with a different ESN ' + str(ie_sdf.ESN.unique())
        elif STCODE:
            match = 'Outside of one of the multiple ranges with a different street type ' + str(ie_sdf.STCODE.unique())
        else:
            match = 'Outside of one of the multiple ranges'
    return diff, match

In [18]:
import timeit

In [19]:
warnings.filterwarnings('ignore')
start = timeit.default_timer()
for key in commonKeys:
    print(key)
    ap_sdf = gdf[gdf.KEY == key]
    hnbr_min = ap_sdf.HouseNbr.values.min()
    hnbr_max = ap_sdf.HouseNbr.values.max()
    ie_sdf = intrado[intrado.KEY==key]
    gdf.loc[gdf.KEY == key, 'RANGE'] = str([[low, high] for low, high in zip(ie_sdf.LOW, ie_sdf.HIGH)])
    ie_sdf.LOW = ie_sdf.LOW.apply(to_int)
    ie_sdf.HIGH = ie_sdf.HIGH.apply(to_int)
    ie_sdf = ie_sdf[((ie_sdf.LOW >= 0) & (ie_sdf.HIGH >= 0))]
    if ie_sdf.shape[0] == 1:
        if hnbr_min >= ie_sdf.LOW.values[0] and hnbr_max <= ie_sdf.HIGH.values[0]:
            gdf.loc[gdf.KEY == key, 'DIFF'] = 1
            gdf.loc[gdf.KEY == key, 'MATCH'] = 'Exactly matched'
        elif hnbr_max > ie_sdf.HIGH.values[0] and hnbr_min >= ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'MATCH'] = 'Outside of house number range in the high value'
            gdf.loc[((gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but the high end in the same street is outside of range'
        elif hnbr_min < ie_sdf.LOW.values[0] and hnbr_max <= ie_sdf.HIGH.values[0]:
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of house number range in the low value'
            gdf.loc[((gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but the low end in the same street is outside of range'
        elif hnbr_max > ie_sdf.HIGH.values[0] and hnbr_min < ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'MATCH'] = 'Outside of house number range in the high value'
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of house number range in the low value'
            gdf.loc[((gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but both high and low ends in the same street are outside of range'
    else:
        gdf.loc[gdf.KEY == key, 'DIFF'] = ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[0])
        gdf.loc[gdf.KEY == key, 'MATCH'] = ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[1])
stop = timeit.default_timer()
print(stop - start)

54__SIUSLAW_RD_WAN
56__HWY 101__FLO
56_S_CANARY_RD_WEL
56__ERHART_RD_FLO
56__SILTCOOS STATION_RD_WEL
56__FIDDLE CREEK_RD_WEL
56__SILTCOOS BEACH ACCESS_RD_FLO
56__LOWER FIDDLE CREEK_RD_WEL
56__LAKE_ST_FLO
56__HWY 101__WEL
54__JERNIGAN ISLAND__WEL
56__3RD_ST_FLO
56__SEAVIEW_LN_FLO
56__BOY SCOUT_RD_FLO
56__MAPLE_DR_FLO
50__HWY 101__FLO
50__BUCKSKIN BOB__FLO
52__BOOTH ISLAND__WEL
50__SALAL_CT_FLO
50__CLOUD NINE_RD_FLO
50__CLEAR LAKE_RD_FLO
50__RUSSELL_DR_FLO
50__KIECHLE ARM_RD_FLO
50__RIO_RD_FLO
50__ERHART_RD_FLO
50__ALDER_CT_FLO
50__WOODLAND_LN_FLO
50__JENSEN_LN_FLO
50__BERRY_LN_FLO
50__MANZANITA_DR_FLO
50__SOUTH COVE_WAY_FLO
50__BRENTWOOD_DR_FLO
50__SOUTH COVE_DR_FLO
50_N_PIONEER_RD_FLO
50__SPRUCE_LN_FLO
50__HILLTOP_DR_FLO
50__PARKWAY_DR_FLO
50__KENDALL_LN_FLO
50__DOROTHY_LN_FLO
50__DARLINGS_LP_FLO
50_S_PIONEER_RD_FLO
50__PACIFIC_AVE_WEL
50__SPRUCE_ST_WEL
50__LAKE_BLVD_WEL
50__SUMMIT_AVE_WEL
50__MEADOW LAKE_WAY_FLO
50__CLEARWATER COVE_WAY_FLO
50__DOONBRAE_LN_FLO
50__SAUTER_DR_FLO
50__LAU

51__SPYGLASS_LN_FLO
51__MARINERS_LN_FLO
51__DUNBAR_WAY_FLO
51__BLUE WATER_CT_FLO
56__TAYLOR_RD_FLO
56__MUNSEL LAKE_RD_FLO
56__STAR VIEW_DR_FLO
56__MARTIN_RD_FLO
56__NORDAHL_RD_FLO
56__LOVELAND_CT_FLO
56__SINGING WOODS_DR_FLO
56__MUNSEL_LN_FLO
51__44TH_ST_FLO
51__52ND_ST_FLO
51__SHERWOOD_LP_FLO
51__ROBIN_LN_FLO
51__45TH_ST_FLO
51__EVERGREEN_LN_FLO
51__PINE TREE_LP_FLO
51__NORTHRIDGE_DR_FLO
51__VERSANT_DR_FLO
51__47TH_ST_FLO
56__MERCER LAKE_RD_FLO
56__BOUCHET_DR_FLO
56__BROWNINGS CORNERS_RD_FLO
56__FRIENDLY ACRES_RD_FLO
56__SUNNY ACRES_RD_FLO
56__LOST_LN_FLO
56__DARR_RD_FLO
56__WINDJAMMER NORTH__FLO
56__WINDJAMMER_WAY_FLO
56__HERMAN CAPE_RD_FLO
56__MCCRAE_RD_FLO
56__SUTTON LAKE_RD_FLO
56__SAVANAH_LN_FLO
56__SUTTON_PL_FLO
56__SHORE CREST_DR_FLO
56__BUCK LAKE_DR_FLO
56__LEVAGE_DR_FLO
56__NORTH_LN_FLO
56__VIEW_DR_FLO
56__LINDA_WAY_FLO
56__SUTTON LAKE_DR_FLO
56__SHERWOOD ISLAND_RD_FLO
56__SOUTH SHORE_DR_FLO
56__SEA LION_DR_FLO
56__OTTER_WAY_FLO
56__SAND DUNE PARK_DR_FLO
56__KNOLL_WAY_FLO
56_

45__TAYLOR_AVE_COT
45_S_3RD_ST_COT
45_S_2ND_ST_COT
45__RIVER WALK_PL_COT
45_E_JEFFERSON_AVE_COT
45_S_5TH_ST_COT
45_E_MONROE_AVE_COT
45_S_DOUGLAS_ST_COT
45_E_QUINCY_AVE_COT
45_S_7TH_ST_COT
45__FILLMORE_AVE_COT
45__LINCOLN_AVE_COT
48__CAMBRIA_PL_COT
48__LINCOLN_AVE_COT
48_S_10TH_ST_COT
48__COOPER_AVE_COT
48_S_12TH_ST_COT
45_E_TAYLOR_AVE_COT
45_S_13TH_ST_COT
45_S_11TH_ST_COT
45__CAROBELLE_CT_COT
45_S_12TH_ST_COT
45_S_16TH_ST_COT
45_S_14TH_ST_COT
45_S_15TH_ST_COT
45__NIKKI_PL_COT
45__KRISTEN_WAY_COT
45__COTTAGE HEIGHTS_LP_COT
45__SAMUEL_DR_COT
45__HILLSIDE_DR_COT
45__GATEWAY_BLVD_COT
45__DUBLIN_LN_COT
45__PIERCE_AVE_COT
45__POLK_AVE_COT
45__PARKS_RD_COT
45__COOPER_AVE_COT
45_S_9TH_ST_COT
45__RED HILLS_PL_COT
48_N_I_ST_COT
45_N_6TH_ST_COT
45_E_MAIN_ST_COT
45_E_WASHINGTON_AVE_COT
45_N_K_ST_COT
45_N_G_ST_COT
45_N_J_ST_COT
45_N_I_ST_COT
45_N_H_ST_COT
45__HOLLY_AVE_COT
45_N_RIVER_RD_COT
45__KALAPUYA_WAY_COT
45__KALAPUYA_CT_COT
45_N_7TH_ST_COT
45_N_8TH_ST_COT
45__CHERRY_CT_COT
45_E_GIBBS_AVE_COT

7__ARLIE_RD_EUG
7__LANCER_LN_EUG
7__SCHNORENBERG_LN_EUG
7__WILD TURKEY_WAY_EUG
7__VINEYARD HILL_DR_EUG
7__BAILEY HILL_RD_EUG
8__BAILEY HILL_RD_EUG
7__RANCH HOUSE_RD_EUG
8__BAILEY HILL_LP_EUG
35__GIMPL HILL_RD_EUG
8__CASTELLOE_AVE_EUG
8__GIMPL_WAY_EUG
8__JARDING_RD_EUG
8__KLOUTZ_RD_EUG
8__RATHBONE_RD_EUG
8__HODSDONSDALE_LN_EUG
8__MCMOROTT_LN_EUG
8__NEEDHAM_RD_EUG
8__LORANE_HWY_EUG
8__BLANTON_RD_EUG
8__WARREN_ST_EUG
8_S_LOUIS_LN_EUG
8__TIMBERLINE_DR_EUG
1__WARREN_ST_EUG
1__GREENTREE_WAY_EUG
1__SUMMIT TERRACE_DR_EUG
1__ASHFORD_DR_EUG
1__BRIGHTON_AVE_EUG
1__SUMMIT SKY_BLVD_EUG
1__TIMBERLINE_DR_EUG
1__BENTLEY_AVE_EUG
1__EAGLE VIEW_DR_EUG
1__TERRACE TRAIL__EUG
1_S_LOUIS_LN_EUG
1__SPOOKY HOLLOW_DR_EUG
1__BAILEY VIEW_DR_EUG
1__LOS ALTOS_LN_EUG
1__TIBURON_LN_EUG
1__COLONY OAKS_DR_EUG
1__TIMBERBROOK_WAY_EUG
1__VISTA HEIGHTS_LN_EUG
1__VISTA HEIGHTS_ALY_EUG
1__BAILEY HILL_RD_EUG
1__CASTELLOE_AVE_EUG
8__CIMARRON_PL_EUG
1__HALLMARK_LN_EUG
1__KISMET_WAY_EUG
1__SUNCREST_AVE_EUG
1__HERALD_LN_EUG
1__TAN

1_W_18TH_AVE_EUG
1__WILLOW CREEK_RD_EUG
1__INTERIOR_ST_EUG
1__PITCHFORD_AVE_EUG
1__OWEN LOOP SOUTH__EUG
1__ISABELLE_ST_EUG
1__ARROWSMITH_ST_EUG
1__TERRY_ST_EUG
1__POWERS_ST_EUG
1_S_DANEBO_AVE_EUG
1__COMMERCE_ST_EUG
1__RENNE_ST_EUG
1__WESTEC_DR_EUG
1__WILLOW CREEK_CIR_EUG
1_S_BERTELSEN_RD_EUG
8__GLEN HAVEN_RD_EUG
1_N_TERRY_ST_EUG
1__THRONE_DR_EUG
1__LEOPOLD_DR_EUG
1__HEATHROW_DR_EUG
1__WELCOME_WAY_EUG
1__DRIFTWOOD_DR_EUG
1__WOODLAND ACRES_LN_EUG
1__FOREST_LN_EUG
1__OAKWOOD_DR_EUG
1__INNSBROOK_WAY_EUG
1__MEHR_AVE_EUG
1__MURIN_ST_EUG
1__CONE_AVE_EUG
1__ROPER_RD_EUG
1__LIVINGSTON_AVE_EUG
1__PARSONS_AVE_EUG
1__DEMARCO_AVE_EUG
1__RABER_RD_EUG
1__ANDREA_AVE_EUG
1__PRANZ_PL_EUG
1__MORELY_LP_EUG
1__GOULD_AVE_EUG
1__ST CHARLES_ST_EUG
1__JACQUELYN_CT_EUG
1__PACIFIC_AVE_EUG
1__OWEN LOOP NORTH__EUG
1__ED CONE_BLVD_EUG
1_W_1ST_AVE_EUG
1__PREMIER_ST_EUG
1__COMMERCIAL_ST_EUG
1_N_DANEBO_AVE_EUG
1__CLOUDBURST_WAY_EUG
1_W_5TH_AVE_EUG
1_W_6TH_AVE_EUG
1__OSCAR_ST_EUG
1_W_7TH_AVE_EUG
1__ROMY_CT_EUG
1__UNTHA

...

27__RYAN_ST_EUG
27__KINGSBURY_AVE_EUG
27__VICTORIA_LN_EUG
27__HAMPSHIRE_LN_EUG
7__MEREDITH_CT_EUG
27__WAKEFIELD_CT_EUG
7__WAKEFIELD_CT_EUG
1__SILVERADO TRAIL__EUG
1__WILLOW SPRINGS_DR_EUG
1__HONEYCOMB_DR_EUG
1__SPRINGWOOD_DR_EUG
1__ALTURA_ST_EUG
1__CHAMPAGNE_LN_EUG
1__WATERSTONE_DR_EUG
1__STONEHAVEN_ST_EUG
1__LARKSMEAD_LN_EUG
1__KINGSBURY_AVE_EUG
1__BERINGER_CT_EUG
1__HAMPSHIRE_LN_EUG
1__STAGS LEAP_CT_EUG
1__SILVER OAK_DR_EUG
1__VICTORIA_LN_EUG
1__CHIMNEY ROCK_LN_EUG
1__MACKIN_AVE_EUG
1__BORESEK_LN_EUG
1__GIPSON_ST_EUG
27__FUTURA_ST_EUG
27__THUNDERBIRD_ST_EUG
27__BRIARS_ST_EUG
27__SCOTTDALE_ST_EUG
27__MAVERICK_AVE_EUG
27__CALUMET_AVE_EUG
27__JILL_AVE_EUG
27__SPRING CREEK_DR_EUG
27__SHANNON_ST_EUG
27__VAN FOSSEN_CT_EUG
27__KILDARE_ST_EUG
27__BRADFORD_CT_EUG
27__SHAMROCK_CT_EUG
27__OROYAN_AVE_EUG
27__DUBLIN_AVE_EUG
27__LIMERICK_AVE_EUG
27__SWAIN_LN_EUG
27__BROTHERTON_AVE_EUG
1__SPRING CREEK_DR_EUG
1__SWAIN_LN_EUG
1__SYMPHONY_DR_EUG
1__SPRING MEADOW_AVE_EUG
1__SEDONA_DR_EUG
1__BRIARS_ST_E

4_N_3RD_ST_CRE
4_E_OREGON_AVE_CRE
4_S_FRONT_ST_CRE
4__HAGENS_CT_CRE
4__HOGANS BRIDGE_CT_CRE
4__TREVINO_RD_CRE
4_W_F_ST_CRE
4_N_MILL_ST_CRE
4_S_1ST_ST_CRE
4__SUNDAY_DR_CRE
4__TASA CREEK_DR_CRE
4__PEBBLE BEACH_DR_CRE
4_N_FRONT_ST_CRE
4__MAGNOLIA_DR_CRE
4__MELTON_RD_CRE
4__VILLAGE_DR_CRE
4_E_PARK_DR_CRE
4_E_HARVEY_RD_CRE
4__MARTIN_RD_CRE
4__HOAGLAND_LN_CRE
4__ALMOND_WAY_CRE
4__COPPER_WAY_CRE
4__CRESWOOD_DR_CRE
4__COBALT_LN_CRE
4__ART LOTT_LN_CRE
4__GRASSLAND_ST_CRE
4__IRONWOOD_LP_CRE
4__HAZELNUT_CT_CRE
4__SANDALWOOD_LP_CRE
4__HONER_LP_CRE
4__HONKERS_ST_CRE
4__OSPREY_LP_CRE
4__ROBIN_CT_CRE
4__CANARY_AVE_CRE
4__CRESWOOD_LP_CRE
4__AVON_ST_CRE
4__CLARK_ST_CRE
4__GREENBRIAR_DR_CRE
4__ADLER_LN_CRE
4__CHRISTMAS TREE_LN_CRE
4__ST ANDREWS_LP_CRE
4__MASTERS_AVE_CRE
4__AUGUSTA_CT_CRE
4__SPYGLASS_CT_CRE
4__RODGERS_RD_CRE
4__ENTERPRISE_RD_CRE
4__RUSSELL OAKS_DR_CRE
4__DANSTROM_RD_CRE
4__BRADFORD_RD_CRE
4__SPRING HILL_RD_CRE
4__BEAR CREEK_RD_CRE
4__WILLS_RD_CRE
4__BEAR MOUNTAIN_RD_CRE
49__WILLS_RD_CRE


30__SENECA_ST_EUG
2__ASPEN_ST_SPR
2_W_D_ST_SPR
2__KELLY_BLVD_SPR
2__HAMILTON_ST_SPR
2__SUNSET_DR_SPR
2_W_H_ST_SPR
2_W_G_ST_SPR
2_W_F_ST_SPR
2_W_E_ST_SPR
2__PRESCOTT_LN_SPR
2__WATER_ST_SPR
2__FAIRWAY_PL_SPR
2__SUMMIT_BLVD_SPR
2__RAINBOW_DR_SPR
2__ISLAND_CT_SPR
2__RIVER HILLS_DR_SPR
2__GRANITE_PL_SPR
2__OKSANNA_ST_SPR
2__WILLACADE_CT_SPR
2__MONTVIEW_WAY_SPR
2__CREST_LN_SPR
2__WOODCREST_DR_SPR
2__CITY VIEW_BLVD_SPR
2__MARILYN_CT_SPR
2__LEVEL_LN_SPR
2__EDGEMONT_WAY_SPR
2__KIEV_ST_SPR
2__RAINTREE_WAY_SPR
2__RIVERVIEW_BLVD_SPR
2__JANNETTE_CT_SPR
2__ISLAND_ST_SPR
2__LUPINE_ST_SPR
2__SALMONBERRY_ST_SPR
1__BOARDWALK_AVE_EUG
1__REGENT_AVE_EUG
1__MCKENNA_DR_EUG
1__MARCHE CHASE_DR_EUG
1_S_GARDEN_WAY_EUG
1__SUNSHINE ACRES_DR_EUG
1__WESTWARD HO_AVE_EUG
1__OXBOW_WAY_EUG
1__FELL_PL_EUG
1__CONESTOGA_WAY_EUG
1__COMMONS_DR_EUG
30_W_QUINALT_ST_SPR
30__LINDEN_AVE_SPR
30__FAIRVIEW_DR_SPR
8_S_GARDEN_WAY_EUG
30__JUNIPER_LN_SPR
30__MENLO_LP_SPR
30__COTTONWOOD_AVE_SPR
30__SEQUOIA_AVE_SPR
30__LODGEPOLE_CT_SPR
30

2__MARTIN LUTHER KING JR_PKWY_SPR
63__MCKENZIE VIEW_DR_EUG
34__SPORTS_WAY_SPR
30__MAPLE ISLAND FARM_RD_SPR
2__SPORTS_WAY_SPR
2__ROYAL CARIBBEAN_WAY_SPR
2__INTERNATIONAL_CT_SPR
34__MCKENZIE VIEW_DR_EUG
63__EGGE_RD_EUG
35__COBURG_RD_EUG
10__EGGE_RD_EUG
35__ARMITAGE_RD_EUG
34__ARMITAGE_RD_EUG
34__SPRAGUE_RD_EUG
15__SPRAGUE_RD_EUG
8_N_GAME FARM_RD_EUG
8__HILLVIEW LANE 2__EUG
8__HILLVIEW LANE 1__EUG
1__HILLVIEW LANE 1__EUG
1__FOX MEADOW_RD_EUG
1__CRAFTSMAN_WAY_EUG
63__DOUTHIT_DR_EUG
24__MCKENZIE VIEW_DR_SPR
34__HARVEST_LN_SPR
34__EGGE_RD_EUG
30__BALDY VIEW_LN_SPR
34__BALDY VIEW_LN_SPR
34__DEADMOND FERRY_RD_SPR
2__RIVERBEND_DR_SPR
2__BALDY VIEW_LN_SPR
2__ST JOSEPH_PL_SPR
30__DONDEA_ST_SPR
30_S_28TH_ST_SPR
30__KINTZLEY_AVE_SPR
30_S_34TH_PL_SPR
30__JASPER_RD_SPR
30__DUKE_ST_SPR
2_S_35TH_ST_SPR
2_S_32ND_PL_SPR
2__KINTZLEY_AVE_SPR
2__DOUGLAS_DR_SPR
2_S_32ND_ST_SPR
2_S_REDWOOD_DR_SPR
2_S_E_CT_SPR
2__OREGON_AVE_SPR
2_S_34TH_ST_SPR
2__ELLIOT_LN_SPR
2__VIRGINIA_AVE_SPR
2__CHEROKEE_DR_SPR
2_S_34TH_PL

23__UPLAND_ST_SPR
23__LUPE_LN_SPR
23__COLLINS_LN_SPR
23__JUNE_LN_SPR
10__WILKINS_RD_EUG
63__COUNTRY VIEW_DR_EUG
34__COUNTRY VIEW_DR_EUG
24__GRAVES_LN_SPR
24__BREWERS BUTTE_RD_SPR
63__COLEMAN-GATEWAY_RD_HAR
63__WILKINS_RD_EUG
24__MCGOWAN CREEK_RD_SPR
24__PEEL_LN_SPR
63__COLEMAN_RD_HAR
63__WEST POINT_RD_HAR
63__VAN DUYN_RD_EUG
63__TRIPLE OAKS_DR_EUG
63__OAK SPRINGS_LN_EUG
63__RIDGEVIEW_RD_EUG
63__BLACKBERRY_LN_EUG
63__COBURGHILLS_DR_EUG
63__SELBY_WAY_EUG
63__DIAMOND RIDGE_LP_EUG
63__MARQUISE_WAY_EUG
63__OAK CREST_RD_EUG
10_E_MAPLE_ST_COB
10_E_THOMAS_ST_EUG
3_N_SKINNER_ST_COB
3_N_COLEMAN_ST_COB
3_N_DIAMOND_ST_COB
3_E_PEARL_ST_COB
3_S_SKINNER_ST_COB
3_E_DELANEY_ST_COB
3_E_DIXON_ST_COB
3_E_MAPLE_ST_COB
3__ROBERTS_RD_COB
3__ROBERTS_CT_COB
3__HUNTLEY_CT_COB
3__VAN DUYN_RD_COB
3_S_COBURG INDUSTRIAL_WAY_COB
3_N_MILLER_ST_COB
3_N_COBURG INDUSTRIAL_WAY_COB
3_E_THOMAS_ST_COB
3_E_LINCOLN_WAY_COB
3_N_SPORES_ST_COB
3__SERENITY_LN_COB
3__RUSTIC_CT_COB
3__SHANE_CT_COB
3_N_EMERALD_ST_COB
3_S_COLEMAN_ST_

In [20]:
gdf[~gdf.RANGE.isna()].shape

(177360, 12)

In [21]:
gdf[gdf.RANGE.isna()].shape

(4862, 12)

In [22]:
gdf.loc[gdf.RANGE.isna(), 'KEY2'] = gdf[gdf.RANGE.isna()].apply(lambda row: row.DI + '_' + row.STREET + '_' +  row.STCODE + '_' +  row.CTCODE, axis=1)

In [23]:
intrado['KEY2'] = intrado.apply(lambda row: row.DI + '_' + row.STREET + '_' +  row.STCODE + '_' +  row.CTCODE, axis=1)

In [24]:
commonKeys2 = [key for key in gdf.KEY2.unique() if key in intrado.KEY2.unique()]

In [25]:
len(commonKeys2)

245

In [26]:
warnings.filterwarnings('ignore')
start = timeit.default_timer()
for key in commonKeys2:
    print(key)
    ap_sdf = gdf[gdf.KEY2 == key]
    hnbr_min = ap_sdf.HouseNbr.values.min()
    hnbr_max = ap_sdf.HouseNbr.values.max()
    ie_sdf = intrado[intrado.KEY2==key]
    gdf.loc[gdf.KEY2 == key, 'RANGE'] = str([[low, high] for low, high in zip(ie_sdf.LOW, ie_sdf.HIGH)])
    ie_sdf.LOW = ie_sdf.LOW.apply(to_int)
    ie_sdf.HIGH = ie_sdf.HIGH.apply(to_int)
    ie_sdf = ie_sdf[((ie_sdf.LOW >= 0) & (ie_sdf.HIGH >= 0))]
    if ie_sdf.shape[0] == 1:
        if hnbr_min >= ie_sdf.LOW.values[0] and hnbr_max <= ie_sdf.HIGH.values[0]:
            gdf.loc[gdf.KEY2 == key, 'DIFF'] = 1
            gdf.loc[gdf.KEY2 == key, 'MATCH'] = 'Matched, but ESN is different, Intrado shows ' + str(ie_sdf.ESN.unique())
        elif hnbr_max > ie_sdf.HIGH.values[0] and hnbr_min >= ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY2 == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY2 == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'MATCH'] = 'Outside of house number range in the high value with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
            gdf.loc[((gdf.KEY2 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY2 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but the high end in the same street is outside of range with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
        elif hnbr_min < ie_sdf.LOW.values[0] and hnbr_max <= ie_sdf.HIGH.values[0]:
            gdf.loc[(gdf.KEY2 == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY2 == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of house number range in the low value with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
            gdf.loc[((gdf.KEY2 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY2 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but the low end in the same street is outside of range with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
        elif hnbr_max > ie_sdf.HIGH.values[0] and hnbr_min < ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY2 == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY2 == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'MATCH'] = 'Outside of house number range in the high value with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
            gdf.loc[(gdf.KEY2 == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY2 == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of house number range in the low value with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
            gdf.loc[((gdf.KEY2 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY2 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but both high and low ends in the same street are outside of range with a different ESN, Intrado shows ' + str(ie_sdf.ESN.unique())
    else:
        gdf.loc[gdf.KEY2 == key, 'DIFF'] = ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[0])
        gdf.loc[gdf.KEY2 == key, 'MATCH'] = ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[1])
stop = timeit.default_timer()
print(stop - start)

_SUMMERBELL_DR_FLO
_OCEAN_BLVD_WEL
_WALKER CREEK_RD_WAN
_WOLF CREEK_RD_VEN
_PANTHER CREEK_RD_VEN
_BAILEY_RD_CRE
_HWY 126__NOT
_TRANSFORMER_RD_WAN
_PATAHA CREEK_RD_WAN
_ALLISON_RD_NOT
_NELSON MOUNTAIN_RD_WAN
_JOSHUA_WAY_NOT
_PENN_RD_WAN
_KEITH_RD_WAN
_SMIGLEY_RD_VEN
_VAUGHN_RD_VEN
_HAYES_RD_VEN
_TERRITORIAL_HWY_VEN
_SLOW DOWN_LN_VEN
_STURTEVANT_DR_VEN
_MATLOCK_LN_VEN
_MARSH_RD_VEN
_MUIRLAND_DR_VEN
_FLECK_RD_VEN
_GREENRIDGE_DR_VEN
_GRAY_RD_VEN
_THOMS_RD_VEN
_WHEATON_LN_VEN
_PRICE_RD_NOT
_KNIGHT_RD_VEN
_HALE_RD_NOT
_CHRISTIAN_RD_VEN
_TORRENCE_RD_NOT
_WILLOW_LN_VEN
_NOTI_LP_NOT
_FIR_ST_NOT
_NOTI_LP_VEN
_SUTTLE_RD_VEN
_ALLEN_RD_VEN
_REDOAK_LN_VEN
_RUSTAD_LN_VEN
_THOMAS_LN_VEN
_GLAZE_RD_VEN
_BOETTGER_LN_VEN
_PENN_RD_VEN
_HAYES_LP_VEN
W_SHEFFLER_RD_NOT
_VINEYARD_LN_VEN
_SANDY_DR_VEN
_REENTS_LN_VEN
_WHITE_RD_VEN
_HWY 126__VEN
_RIDICULOUS_RD_VEN
_BOLTON HILL_RD_VEN
_SERTIC_RD_VEN
_CHICKADEE_LN_VEN
_DAWN_CT_VEN
_9TH_ST_VEN
_10TH_ST_VEN
_SHERWOOD_ST_VEN
_8TH_ST_VEN
_6TH_ST_VEN
_FERNDALE_CT_VEN
_T

In [27]:
gdf[gdf.RANGE.isna()].shape

(301, 13)

In [28]:
gdf.loc[gdf.RANGE.isna(), 'KEY3'] = gdf[gdf.RANGE.isna()].apply(lambda row: str(row.ESN) + '_' + row.DI + '_' + row.STREET + '_' +  row.CTCODE, axis=1)

In [29]:
intrado['KEY3'] = intrado.apply(lambda row: str(row.ESN) + '_' + row.DI + '_' + row.STREET + '_' +  row.CTCODE, axis=1)

In [30]:
commonKeys3 = [key for key in gdf.KEY3.unique() if key in intrado.KEY3.unique()]

In [31]:
len(commonKeys3)

7

In [32]:
warnings.filterwarnings('ignore')
start = timeit.default_timer()
for key in commonKeys3:
    print(key)
    ap_sdf = gdf[gdf.KEY3 == key]
    hnbr_min = ap_sdf.HouseNbr.values.min()
    hnbr_max = ap_sdf.HouseNbr.values.max()
    ie_sdf = intrado[intrado.KEY3==key]
    gdf.loc[gdf.KEY3 == key, 'RANGE'] = str([[low, high] for low, high in zip(ie_sdf.LOW, ie_sdf.HIGH)])
    ie_sdf.LOW = ie_sdf.LOW.apply(to_int)
    ie_sdf.HIGH = ie_sdf.HIGH.apply(to_int)
    ie_sdf = ie_sdf[((ie_sdf.LOW >= 0) & (ie_sdf.HIGH >= 0))]
    if ie_sdf.shape[0] == 1:
        if hnbr_min >= ie_sdf.LOW.values[0] and hnbr_max <= ie_sdf.HIGH.values[0]:
            gdf.loc[gdf.KEY3 == key, 'DIFF'] = 1
            gdf.loc[gdf.KEY3 == key, 'MATCH'] = 'Matched, but the street type is different, Intrado shows ' + str(ie_sdf.STCODE.unique())
        elif hnbr_max > ie_sdf.HIGH.values[0] and hnbr_min >= ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY3 == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY3 == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'MATCH'] = 'Outside of house number range in the high value with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
            gdf.loc[((gdf.KEY3 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY3 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but the high end in the same street is outside of range with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
        elif hnbr_min < ie_sdf.LOW.values[0] and hnbr_max <= ie_sdf.HIGH.values[0]:
            gdf.loc[(gdf.KEY3 == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY3 == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of house number range in the low value with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
            gdf.loc[((gdf.KEY3 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY3 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but the low end in the same street is outside of range with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
        elif hnbr_max > ie_sdf.HIGH.values[0] and hnbr_min < ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY3 == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY3 == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'MATCH'] = 'Outside of house number range in the high value with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
            gdf.loc[(gdf.KEY3 == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY3 == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of house number range in the low value with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
            gdf.loc[((gdf.KEY3 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'DIFF'] = 1
            gdf.loc[((gdf.KEY3 == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'MATCH'] = 'Matched, within the house number range, but both high and low ends in the same street are outside of range with a different street type, Intrado shows ' + str(ie_sdf.STCODE.unique())
    else:
        gdf.loc[gdf.KEY3 == key, 'DIFF'] = ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[0])
        gdf.loc[gdf.KEY3 == key, 'MATCH'] = ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[1])
stop = timeit.default_timer()
print(stop - start)

45__TAYLOR_COT
1__DEERBRUSH_EUG
1__UNIVERSITY_EUG
1__COUNTRY CLUB_EUG
1__OAKWAY_EUG
8__PARK_EUG
1__PARK_EUG
0.34271350000005896


In [33]:
gdf[~gdf.RANGE.isna()].shape

(181984, 14)

In [34]:
gdf[gdf.RANGE.isna()].shape

(238, 14)

In [35]:
gdf.loc[gdf.RANGE.isna(), 'DIFF'] = 0
gdf.loc[gdf.RANGE.isna(), 'MATCH'] = 'No match'

In [37]:
gdf.loc[gdf.ESN == 39, 'DIFF'] = 5
gdf.loc[gdf.ESN == 39, 'MATCH'] = 'Missing ESN in the Intrado extract'

In [38]:
gdf.loc[(gdf.DIFF==0) & (gdf.STREET == 'CHESHIRE'), 'MATCH'] = 'Matched without the direction in Intrado'
gdf.loc[(gdf.DIFF==0) & (gdf.STREET == 'CHESHIRE'), 'DIFF'] = 1

In [39]:
gdf.MATCH.unique()

array(['Exactly matched', 'Matched in one of the multiple ranges',
       'Outside of one of the multiple ranges',
       'Matched, but ESN is different, Intrado shows [51]',
       'Matched, within the house number range, but the low end in the same street is outside of range',
       'Matched, but ESN is different, Intrado shows [50]',
       'Outside of house number range in the low value',
       'Matched, within the house number range, but the high end in the same street is outside of range',
       'Outside of house number range in the high value',
       'Outside of the multiple house number ranges in the high value',
       'No match', 'Missing ESN in the Intrado extract',
       'Matched, but ESN is different, Intrado shows [7]',
       "Outside of house number range in the low value with a different street type, Intrado shows ['AVE']",
       'Matched, but ESN is different, Intrado shows [45]',
       'Outside of house number range in the high value with a different ESN, Intr

In [40]:
ngdf = gdf.drop(['KEY', 'KEY2', 'KEY3'], axis=1)

In [41]:
ngdf.to_file(r'G:\projects\Address_Points\9-1-1_Net\AddressPoints.shp')

In [42]:
ngdf.DIFF.unique()

array([1., 4., 3., 2., 0., 5.])